Copyright (c) 2024, NVIDIA CORPORATION.  All rights reserved.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

MONAI Example adopted from https://github.com/Project-MONAI/tutorials/blob/main/2d_classification/monai_101.ipynb

Copyright (c) MONAI Consortium  
Licensed under the Apache License, Version 2.0 (the "License");  
you may not use this file except in compliance with the License.  
You may obtain a copy of the License at  
&nbsp;&nbsp;&nbsp;&nbsp;http://www.apache.org/licenses/LICENSE-2.0  
Unless required by applicable law or agreed to in writing, software  
distributed under the License is distributed on an "AS IS" BASIS,  
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.  
See the License for the specific language governing permissions and  
limitations under the License.

# MONAI 101 tutorial with Federated Learining

In this tutorial, we will introduce how simple we can convert the MONAI 101 tutorial into a federated learning training script

These steps will be included in this tutorial, and each of them will take only a few lines of code:
- Dataset download
- Data pre-processing
- Define a DenseNet-121 and run training
- Check the results on test dataset

This tutorial will use about 7GB of GPU memory and 10 minutes to run.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Project-MONAI/tutorials/blob/main/2d_classification/monai_101.ipynb)

## Setup environment

In [1]:
!python -c "import monai" || pip install -q "monai-weekly[ignite, tqdm]"
!pip install -r requirements.txt
# install the latest nvflare
!pip install -e ../../../..

Obtaining file:///media/hroth/NVIDIA/home_old/hroth/Code2/nvflare/monai_flywheel
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for nvflare-nightly (pyproject.toml) ... done
  Created wheel for nvflare-nightly: filename=nvflare_nightly-2.4.0rc7+112.gd6f1f587.dirty-0.editable-py3-none-any.whl size=10619 sha256=f6e9c90525bd5ecf6a0fe27c43aa450c7f673a4771c0ee3bacafa3f557543aea
  Stored in directory: /tmp/pip-ephem-wheel-cache-qgc0s0xz/wheels/15/f0/59/04a976ff344afbdbbd7bb194f92e23f23577de3f84fd53b9d8
Successfully built nvflare-nightly
  Attempting uninstall: nvflare-nightly
    Found existing installation: nvflare-nightly 2.4.0rc7+111.g01453824.dirty
    Uninstalling nvflare-nightly-2.4.0rc7+111.g01453824.dirty:
      Successfully uninstalled nvflare-nightly-2.4.0rc7+111.g01453824.dirty


## Configure the NVFlare job templates folder

In [2]:
!nvflare config -jt ./job_templates
!nvflare job list_templates


The following job templates are available: 

------------------------------------------------------------------------------------------------------------------------
  name                 Description                                                  Controller Type   Execution API Type     
------------------------------------------------------------------------------------------------------------------------
  fedavg_mednist       FedAvg using MONAI with in_process Client API                server            client_api             
------------------------------------------------------------------------------------------------------------------------


We will use the in-process client API, we choose the [sag_pt in_proc job template](../../../job_templates/sag_pt_in_proc) and run the following command to create the job:

In [3]:
!nvflare job create -force -j ./jobs/fedavg_mednist -w fedavg_mednist -sd ./code/. \
    -f config_fed_client.conf app_script=monai_mednist_train.py \
    -f config_fed_server.conf model_class_path=monai.networks.nets.densenet121 \
    -f config_fed_server.conf spatial_dims=2 \
    -f config_fed_server.conf in_channels=1 \
    -f config_fed_server.conf out_channels=6 \
    -f config_fed_server.conf num_rounds=10


The following are the variables you can change in the template

---------------------------------------------------------------------------------------------------------------------------------------
                                                                                                                                       
  job folder: ./jobs/fedavg_mednist                                                                                                      
                                                                                                                                       
---------------------------------------------------------------------------------------------------------------------------------------
  file_name                      var_name                       value                               component                          
---------------------------------------------------------------------------------------------------------------------

Then we can run it using the NVFlare Simulator:

In [4]:
!nvflare simulator -n 2 -t 2 ./jobs/fedavg_mednist -w fedavg_workspace

2024-04-26 16:01:16,820 - SimulatorRunner - INFO - Create the Simulator Server.
2024-04-26 16:01:16,822 - CoreCell - INFO - server: creating listener on tcp://0:43925
2024-04-26 16:01:16,839 - CoreCell - INFO - server: created backbone external listener for tcp://0:43925
2024-04-26 16:01:16,839 - ConnectorManager - INFO - 2598417: Try start_listener Listener resources: {'secure': False, 'host': 'localhost'}
2024-04-26 16:01:16,840 - nvflare.fuel.f3.sfm.conn_manager - INFO - Connector [CH00002 PASSIVE tcp://0:26178] is starting
2024-04-26 16:01:17,341 - CoreCell - INFO - server: created backbone internal listener for tcp://localhost:26178
2024-04-26 16:01:17,342 - nvflare.fuel.f3.sfm.conn_manager - INFO - Connector [CH00001 PASSIVE tcp://0:43925] is starting
2024-04-26 16:01:17,414 - nvflare.fuel.hci.server.hci - INFO - Starting Admin Server localhost on Port 53755
2024-04-26 16:01:17,414 - SimulatorRunner - INFO - Deploy the Apps.
2024-04-26 16:01:17,417 - SimulatorRunner - INFO - Crea